# Install Dependecies

In [3]:
!pip install --upgrade pip

In [2]:
!pip install -q torch
!pip install -q accelerate  bitsandbytes transformers
!pip install youtube-transcript-api langchain pypdf faiss-cpu google-search-results sentence-transformers langchainhub

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 14.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 kB 10.0 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=8baf53eeafa1de12b968cf8d9a9b1f6d465917e94b5224f48abe101c84712884
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully bui

# Import library

In [67]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM , AutoTokenizer , BitsAndBytesConfig , pipeline ,VitsModel
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from getpass import getpass
import os
from huggingface_hub import login
from IPython.display import Audio
import torch

In [68]:
login()

# Mistral LLm

In [4]:
def uploadTokenizer():
  tokenizer=AutoTokenizer.from_pretrained(model_repo)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.pad_token_id = tokenizer.eos_token_id
  tokenizer.padding_side = "right"

  return tokenizer

In [5]:
def uploadModel(tokenizer):
  compute_dtype = getattr(torch, "float16")
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    )
  model=AutoModelForCausalLM.from_pretrained(model_repo,
                             quantization_config=bnb_config)
  llm=pipeline(
    model=model,
    task="text-generation",
    tokenizer=tokenizer,
    temperature=0.6,
    return_full_text=True,
    do_sample=True,
    max_new_tokens=1000,
    )
  mistral = HuggingFacePipeline(pipeline=llm)
  return mistral

In [6]:
model_repo="mistralai/Mistral-7B-Instruct-v0.1"
tokenizer=uploadTokenizer()
mistral=uploadModel(tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

 # OpenChat Model

In [4]:
#model_repo="openchat/openchat-3.5-0106"
#tokenizer=uploadTokenizer()
#openChat=uploadModel(tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.

  warnings.warn(


# YT video Transcript

In [7]:
def getVidoTranscript(url):
  video_id = url.replace('https://www.youtube.com/watch?v=', '')
  transcript = YouTubeTranscriptApi.get_transcript(video_id , languages=["en"])
  output=''
  for x in transcript:
    sentence = x['text']
    output += f' {sentence}\n'
  return output

# RAG

In [8]:
def split_text(text):
  text_splitter=RecursiveCharacterTextSplitter(
      chunk_size=100,
      chunk_overlap=20,
      length_function=len,
      is_separator_regex=False,
      )
  text=text_splitter.split_text(text)
  return text
def retiverFaiss(text):
  db=FAISS.from_texts(
      text,
      HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    )
  retiver=db.as_retriever()
  return retiver


In [36]:
def getPrompt(template):
    prompt=PromptTemplate(
        input_variables=["question"],
        template=template
    )
    return prompt
    
template="""Instructions :[INST] Extract knowledge from YouTube transcripts, answering user questions without seeing the video.
Rules:
Transcript-only: No peeking at videos!
Contextual Comprehension: Unravel the meaning and tone.
Information Guru: Deliver accurate, comprehensive answers.
Language Ninja: Make connections, infer, and impress!
here is the transcript you need :
{transcript}

Here is the user's question :
{question}
[/INST]
"""

In [37]:
def getCompletion(prompt,retiver,model,question):
  chain=LLMChain(prompt=prompt,llm=model)
  rag_chain = ({
    "transcript":retiver,
    "question":RunnablePassthrough()
    }|chain
  )
  result=rag_chain.invoke(question)
  return result["text"]


# Audio Assistant

In [90]:
VoiceModel=pipeline(
    task="text-to-speech",
    model="facebook/mms-tts-eng"
)
def getResultSpeech(result):
    output=VoiceModel(result)
    return output    

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [70]:
text=getVidoTranscript("https://www.youtube.com/watch?v=ll5LY7wI_Xc&t=729s")

In [71]:
text=split_text(text)
retiver=retiverFaiss(text)

In [92]:
result=getCompletion(getPrompt(template),retiver,mistral,question="Can you summurize this video ?")

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [93]:
output=getResultSpeech(result)
Audio(output["audio"], rate=output["sampling_rate"])

In [88]:
result=getCompletion(getPrompt(template),retiver,mistral,question="Can Ai perform better than doctors ?")

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [91]:
output=getResultSpeech(result)
Audio(output["audio"], rate=output["sampling_rate"])